In [1]:
%matplotlib notebook
import matplotlib.animation as ani
import matplotlib.colors as colors
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from planetThermo import pco2
from planetThermo import tco2
import scipy.linalg
from scipy.interpolate import pchip
import statsmodels.api as sm
import pickle
plt.rc('font', family='serif')
plt.rcParams['font.weight']= 'heavy'
plt.rcParams.update({'font.size': 12})

In [14]:
#I've saved the binning routines to a module python file

#List of binning methods:

#Zonal mean binning with Altitude z-axis: 
from MCS_modules import Alt_Lat_binning

#Zonal mean binning with Pressure z-axis:    
from MCS_modules import Pres_Lat_binning

##import the radius profile functions:
from MCS_modules import radius_profile
from MCS_modules import Ls_radius_dist
from MCS_modules import Cloud_Model_CN
#
##import the water build up functions
from MCS_modules import All_lat_snowfall
from MCS_modules import N_Water_buildup as Water_buildup
from MCS_modules import N_CO2_buildup as CO2_buildup

##import data manipulation functions
from MCS_modules import Ls_filler
from MCS_modules import remove_outliers

In [3]:
#reading from parquet file
#df = pd.read_parquet('MCS_MY29_20071209_20081225.parquet.gzip'); yr = 29 #MY 29 
#df = pd.read_parquet('MCS_MY30_20091026_20101112.parquet.gzip'); yr = 30 #MY 30 
#df = pd.read_parquet('MCS_MY31_20110913_20120731.parquet.gzip'); yr = 31 #MY 31 
#df = pd.read_parquet('MCS_MY32_20130713_20150618.parquet.gzip'); yr = 32 #MY 32
#df = pd.read_parquet('MCS_MY33_20150618_20170505.parquet.gzip'); yr = 33 #MY 33
#df = pd.read_parquet('MCS_MY34_20171028_20190322.parquet.gzip'); yr = 34  #MY 34
#df = pd.read_parquet('MCS_MY35_20190322_20200408.parquet.gzip'); yr = 35  #MY 35

In [4]:
# create a new dataframe that only includes the polar vortex of the north pole region (North Polar Vortex)
SPV = df[(df.LS <= 180)&(df.LS >=0)&(df.ALTITUDE <= 60)&(df.LATITUDE <= -65)&(df.LATITUDE >= -87)]
#resetting index of new dataframe
SPV = SPV.reset_index()
SPV = SPV.drop(['index'],axis = 1)

#calculate the frost point for each entry
SPV['T_frost'] = tco2(0.95*SPV.PRESSURE)

# create a new dataframe that only includes the CO2 measurements,, which is anywhere where the recorded 
#temp is equal to or less than the frost point
SPVC = SPV[(SPV.TEMPERATURE <= SPV.T_frost+5)]
#resetting index of new dataframe
SPVC = SPVC.reset_index()
SPVC = SPVC.drop(['index'],axis = 1)

#bin the data
LatV,AltV, Vtemp = Alt_Lat_binning(SPVC,'temp',1)
LatV,AltV, co2ice = Alt_Lat_binning(SPVC,'dust',1) #here, dust is actually co2 since we've seperated out the co2 from the dust data
LatV,AltV, Vh2oice = Alt_Lat_binning(SPVC,'h2oice',1)
latV,AltV, Vpress = Alt_Lat_binning(SPVC,'pressure',1)
names = list(Vtemp.keys())
#convert Ls names list into integers
intnames = np.array(names).astype(np.float).astype(np.int)

In [6]:
## 1 extracting the full WATER no. density profile 

#for indexing purposes
names = list(Vtemp.keys())

#define constants/inputs (using km as base length)
Qh = 0.7467 #efficiency of extinction for water ice
ah = 4e-6 #m molecule cross section 
rhoh = 1000 #individual particle mass density of water ice (Cotton et al 2013 JRMS)

R_m = 3.3895e6 #radius of Mars in m

h = 1000 #m # vertical height of 1 pixel

mu = 8.5e-6 #(kg/ m s) OR (Pa s) # molecular viscosity of atmosphere at 150 K

G = 6.67408e-11 #m3/kg s  # gravitational constant

Kb = 1.38064852e-23 #m2 kg/ s2 K #boltzmann constant

M_m = 6.39e23 #kg # mass of Mars in Kgrams

gr = 3.711 #m/s²

Tav = 140 #K #average temp at the poles

#create empty dictionaries to store results
N_H2Odensity = dict()


#Scale the altitude to be in meters:
SAltV = AltV*1e3
#quick note on how the dictionary data can be accessed for example the dust matrix:
#h2oice[names[frame(ls) index]][altitude index][latitude index]
#this is important to mention since I will need to access the info saved in 
#the binning dictionary to perform this analysis

#loop through each entry in the co2ice dict
for i in tqdm(range(len(Vh2oice))):
    # redefine the array of co2ice value
    T = Vtemp[names[i]] #K
    P = Vpress[names[i]]#kg/m s2 OR Pa
    H = Vh2oice[names[i]]*1e-3 #m^-1 (multiply by 1e-3 to convert from km-1 to m-1)
    
    #step 1: convert the H2O ice opacities to optical depth
    H = H*h
    #step 2: calculate the no. density of H2O ice particles in each pixel
    
    N_h = H/(Qh*np.pi*(ah**2)*h) 
    
    #step 2.5 calculate the scale height of the atmosphere:
    polartemp = np.nanmean(np.ma.masked_where(T == 0, T))
    Scale_H = (Kb*polartemp)/(44*1.67e-27*gr)
    
    #step 3
    #finding the index of the bottom most non nan entry
    for j in range(len(LatV)-1):
        #first find the indices of all non nan entries in that column
        nonnan = np.argwhere(~np.isnan(N_h[:,j])&~(N_h[:,j]==0))
        
        #don't do anything for columns with all nan entries
        if nonnan.size == 0:
            pass
        #for columns with non nan entries, extrapolate to the bottom using the bottom most non nan entry
        else:
            for m in range(len(nonnan)-1):
                if nonnan[m+1] - nonnan[m] >= 5:
                    delidx = int(nonnan[m+1])
                    N_h[delidx:,j] = 0
                    P[delidx:,j] = np.nan
                else:
                    continue
            
            bottomindex = np.int(np.min(nonnan))
            
            #take the av of the two bottom most entries for the extrapolation
            if N_h[bottomindex+1][j] > 0: 
                AvN_h = (N_h[bottomindex][j] + N_h[bottomindex+1][j])/2
                AvP = (P[bottomindex][j] + P[bottomindex+1][j])/2
                #set the bottom most entry to be the av
                N_h[bottomindex][j] = AvN_h 
                P[bottomindex][j] = AvP
            else:
                AvN_h = N_h[bottomindex][j]
                AvP = P[bottomindex][j]
                
            #using n = n_o exp(-z/H) find the no. density at each altitude below the bottom most entry
            for z in range(bottomindex):
                #using n = n_o exp(-z/H) find the no. density at each altitude below the bottom most entry
                
                N_sh = (AvN_h/(np.exp(-SAltV[bottomindex]/Scale_H)))*(np.exp(-SAltV[z]/Scale_H))
                Pressures = (AvP/(np.exp(-SAltV[bottomindex]/Scale_H)))*(np.exp(-SAltV[z]/Scale_H))
                
                N_h[z][j] = N_sh
                P[z][j] = Pressures
                   
    
    #remove all nan values
    N_h[np.isnan(N_h)] = 0  
    
    N_H2Odensity[names[i]] = N_h

100%|██████████| 179/179 [00:00<00:00, 394.96it/s]


In [7]:
## 1 extracting the full CO2 no. density profile as

#for indexing purposes
names = list(Vtemp.keys())

#define constants/inputs (using km as base length)
Qc = 3.0249 #efficiency of extinction for CO2
ac = 1e-5 #m particle cross section
rhoc = 1600 #kg/m3 #solid density of CO2 ice (Find reference)

R_m = 3.3895e6 #radius of Mars in m

h = 1000 #m # vertical height of 1 pixel

mu = 8.5e-6 #(kg/ m s) OR (Pa s) # molecular viscosity of atmosphere at 150 K

G = 6.67408e-11 #m3/kg s  # gravitational constant

Kb = 1.38064852e-23 #m2 kg/ s2 K #boltzmann constant

dc = 3.3e-10 #m CO2 molecule diameter

M_m = 6.39e23 #kg # mass of Mars in Kgrams

gr = 3.711 #m/s²

m_p = (4/3)*rhoc*np.pi*(ac**3) #average mass of co2 particle of size ac

Tav = 140 #K #average temp at the poles

#create empty dictionaries to store results
N_density = dict() #no density of CO2 particles
Settle_v = dict() #settling velocity of the particles
Temp = dict() #updated temperature dictionary with gaps filled (will be used in IR flux calculations later)
Press = dict() #updated pressure dictionary with gaps filled (will be used in IR flux calculation later)
Frost = dict() #calculated CO2 frost temperature 

#Scale the altitude to be in meters:
SAltV = AltV*1e3
#quick note on how the dictionary data can be accessed for example the dust matrix:
#h2oice[names[frame(ls) index]][altitude index][latitude index]
#this is important to mention since I will need to access the info saved in 
#the binning dictionary to perform this analysis

#loop through each entry in the co2ice dict
for i in tqdm(range(len(co2ice))):
    # redefine the array of co2ice value
    T = Vtemp[names[i]] #K
    P = Vpress[names[i]]#kg/m s2 OR Pa
    C = co2ice[names[i]]*1e-3 #m^-1 (multiply by 1e-3 to convert from km-1 to m-1)
    Tfrost = tco2(0.95*P) #K frost temp of CO2 
    
    #step 1: convert the co2 ice opacities to optical depth
    C = C*h
    #step 2: calculate the no. density of CO2 ice particles in each pixel
    
    N_c = 2.3*C/(Qc*np.pi*(ac**2)*h) #scaling the opacity CO2/dust ratio = 2.3 (source: Dave's work for Paul and Carlos's paper)
    
    #step 2.5 calculate the scale height of the atmosphere:
    polartemp = np.nanmean(np.ma.masked_where(T == 0, T))
    Scale_H = (Kb*polartemp)/(44*1.67e-27*gr)
    #step3: calculate mass flux due to gravitational settling 
    
    #empty array to store the settling velocity w
    w = np.zeros(shape =(C.shape[0],C.shape[1]))
    
    #finding the index of the bottom most non nan entry
    for j in range(len(LatV)-1):
        #first find the indices of all non nan entries in that column
        nonnan = np.argwhere(~np.isnan(N_c[:,j])&~(N_c[:,j]==0))
        #don't do anything for columns with all nan entries
        if nonnan.size == 0:
            pass
        #for columns with non nan entries, extrapolate to the bottom using the bottom most non nan entry
        else:
            # first remove outliers from the top of atmosphere (if 5 or more bins away from column)
            for m in range(len(nonnan)-1):
                if nonnan[m+1] - nonnan[m] >= 5:
                    delidx = int(nonnan[m+1])
                    N_c[delidx:,j] = 0
                    P[delidx:,j] = np.nan
                else:
                    continue
            
            bottomindex = np.int(np.min(nonnan))
        
            #take the av of the bottom two points for the extrapolation
            if N_c[bottomindex+1][j] > 0: 
                AvN_c = (N_c[bottomindex][j] + N_c[bottomindex+1][j])/2
                AvP = (P[bottomindex][j] + P[bottomindex+1][j])/2
                #set the bottom most entry to be the av
                N_c[bottomindex][j] = AvN_c
                P[bottomindex][j] = AvP
            else:
                AvN_c = N_c[bottomindex][j]
                AvP = P[bottomindex][j]
            
            #using n = n_o exp(-z/H) find the no. density at each altitude below the bottom most entry
            for z in range(bottomindex):
                #using n = n_o exp(-z/H) find the no. density at each altitude below the bottom most entry
                
                
                N_sc = (AvN_c/(np.exp(-SAltV[bottomindex]/Scale_H)))*(np.exp(-SAltV[z]/Scale_H))
                Pressures = (AvP/(np.exp(-SAltV[bottomindex]/Scale_H)))*(np.exp(-SAltV[z]/Scale_H))
                
                N_c[z][j] = N_sc
                P[z][j] = Pressures
                
                frost_temp = tco2(0.95*Pressures) #fill in frost temp of CO2 for those lower parts using new calculated pressure 
                
                Tfrost[z][j] = frost_temp
                #also fill in the temperarture for those missing layers, if the temp of the bottom most entry
                #is greater than the frost temp of CO2, use that. Otherwise use the Tfrost of CO2 cause that is our minimum
                if T[bottomindex][j] >= frost_temp:
                    T[z][j] = T[bottomindex][j]
                else:
                    T[z][j] = frost_temp
                
                   
    #find the settling velocity (need this as an initial value for the finite diff method)
    for z in range(len(SAltV)-1):
        for j in range(len(LatV)-1):
            
            #step 3c:  calculate the settling velocity using Stokes–Cunningham law for spherical particles 
            #g = (G*M_m)/((R_m+ SAltV[z])**2) #mars gravity at certain altitude
            if P[z][j] == 0:
                w[z][j] = 0
            else:
                mfp = (Kb*Tav)/(np.sqrt(2)*np.pi*(dc**2)*P[z][j]) #mean free path in air
                w[z][j] = ((2*rhoc*gr*(ac**2))/(9*mu))*(1+ (mfp/ac)) #m/s
            
    
    
    #remove all nan values
    N_c[np.isnan(N_c)] = 0  
    w[np.isnan(w)] = 0 
    T[np.isnan(T)] = 0 
    P[np.isnan(P)] = 0 
    Tfrost[np.isnan(Tfrost)] = 0 
    
    
    N_density[names[i]] = N_c
    Settle_v[names[i]] = w
    Temp[names[i]] = T
    Frost[names[i]] = Tfrost
    Press[names[i]] = P

100%|██████████| 179/179 [00:04<00:00, 35.84it/s]


In [15]:
#PERFORM ANALYSIS FOR ENTIRE WINTER SEASON

#Set up the total time to run each settling run
T = 60*60*4 #sec
#time step 
dt = 60 #sec
#altitude step
dz = 0.5*1e3 #m
#diffusion coefficient
K = 1e3 #m2/s
# Find the buildup of water for the entire winter season
#assume that the snapshot of one LS is representative cloud profile for that Ls
#assuming the cloud is depleted in approx 4 hours, multiply the amnt of water by the number of 4 hour periods in each LS
# one Ls = approx 11.25 4 hour periods 
#for 10 microns, its never depleted so just multiply by how many 24 hour periods there are and run for 24 hours
MULTY = 11.25
#create empty matrix to store water deposited per latitude, evolving through time:
wat_dep = np.zeros(shape = (len(names),LatV.size-1)) #first index is time, second is lat
watmass = np.zeros(shape = (len(names),LatV.size-1)) #first index is time, second is lat
#create empty matrix to store CO2 deposited per latitude, evolving through time:
dry_dep = np.zeros(shape = (len(names),LatV.size-1)) #first index is time, second is lat
drymass = np.zeros(shape = (len(names),LatV.size-1)) #first index is time, second is lat
#create empty matrix to store CO2 particle radius evolving through time:
CO2_radii = np.zeros(shape = (len(names))) #first index is time, second is lat

for i in tqdm(range(len(names))):
    if N_density[names[i]].sum() ==0:
        wat_dep[i,:] = 0
        dry_dep[i,:] = 0
        watmass[i,:] = 0
        drymass[i,:] = 0
        CO2_radii[i] = 0
    else:
        New_density,N_time_evolution,Sed_evol_lat,Radii= All_lat_snowfall(N_density[names[i]],Temp[names[i]],Frost[names[i]],Press[names[i]],T,dz,dt,K,LatV)
        Sed_evol_lat[np.isnan(Sed_evol_lat)] = 0
        Sed_evol_lat[Sed_evol_lat<=0] = 0
        Sed_evol_lat[Sed_evol_lat>=1e7] = 0
        H2O_cum, H2O_mass = Water_buildup(Sed_evol_lat,LatV,dt)
        CO2_cum, CO2_mass = CO2_buildup(Sed_evol_lat,Radii,LatV,dt)
        wat_dep[i,:] = H2O_cum[-1,:]*MULTY
        dry_dep[i,:] = CO2_cum[-1,:]*MULTY
        watmass[i,:] = H2O_mass[-1,:]*MULTY
        drymass[i,:] = CO2_mass[-1,:]*MULTY
        CO2_radii[i] = np.nanmedian(Radii[0,:])
    
Water_deposit = wat_dep[:,:]
CO2_deposit = dry_dep[:,:]
Mass_deposit = watmass[:,:]
CMass_deposit = drymass[:,:]

100%|██████████| 179/179 [01:16<00:00,  2.34it/s]


In [ ]:
##MY 29;
#missing_Ls = [0, 32, 107, 140]
#dups = [5,1,1,1]

##MY 30;
#missing_Ls = [0, 149]
#dups = [24,1]

##MY 31;
#missing_Ls = [26, 70, 148]
#dups = [1,1,33]

##MY 32;
#missing_Ls = [1, 100, 110]
#dups = [2,1,8]

##MY 33;
#missing_Ls = [101, 142, 172]
#dups = [3,1,5]

##MY 34;
#missing_Ls = [0, 131, 144]
#dups = [80,7,2]

##MY 35;
#missing_Ls = [153]
#dups = [4]

In [ ]:
#Fill in the gaps
Water_deposit = Ls_filler(Water_deposit, missing_Ls, LatV, intnames, dups)
CO2_deposit = Ls_filler(CO2_deposit, missing_Ls, LatV, intnames, dups)
Mass_deposit = Ls_filler(Mass_deposit, missing_Ls, LatV, intnames, dups)
CMass_deposit = Ls_filler(CMass_deposit, missing_Ls, LatV, intnames, dups)
Water_cum = np.cumsum(Water_deposit, axis = 0)
Mass_cum = np.cumsum(Mass_deposit, axis = 0)
DryIce_cum = np.cumsum(CO2_deposit, axis = 0)
CMass_cum = np.cumsum(CMass_deposit, axis = 0)

In [ ]:
##save it for that specfic year
##MY29
#Water_deposit29 = Water_deposit
#CO2_deposit29 = CO2_deposit
#Mass_deposit29 = Mass_deposit
#CMass_deposit29 = CMass_deposit
#Water_cum29 = Water_cum
#Mass_cum29 = Mass_cum
#DryIce_cum29 = DryIce_cum
#CMass_cum29 = CMass_cum

###MY30
#Water_deposit30 = Water_deposit
#CO2_deposit30 = CO2_deposit
#Mass_deposit30 = Mass_deposit
#CMass_deposit30 = CMass_deposit
#Water_cum30 = Water_cum
#Mass_cum30 = Mass_cum
#DryIce_cum30 = DryIce_cum
#CMass_cum30 = CMass_cum

##MY31
#Water_deposit31 = Water_deposit
#CO2_deposit31 = CO2_deposit
#Mass_deposit31 = Mass_deposit
#CMass_deposit31 = CMass_deposit
#Water_cum31 = Water_cum
#Mass_cum31 = Mass_cum
#DryIce_cum31 = DryIce_cum
#CMass_cum31 = CMass_cum

##MY32
#Water_deposit32 = Water_deposit
#CO2_deposit32 = CO2_deposit
#Mass_deposit32 = Mass_deposit
#CMass_deposit32 = CMass_deposit
#Water_cum32 = Water_cum
#Mass_cum32 = Mass_cum
#DryIce_cum32 = DryIce_cum
#CMass_cum32 = CMass_cum

##MY33
#Water_deposit33 = Water_deposit
#CO2_deposit33 = CO2_deposit
#Mass_deposit33 = Mass_deposit
#CMass_deposit33 = CMass_deposit
#Water_cum33 = Water_cum
#Mass_cum33 = Mass_cum
#DryIce_cum33 = DryIce_cum
#CMass_cum33 = CMass_cum

##MY34
#Water_deposit34 = Water_deposit
#CO2_deposit34 = CO2_deposit
#Mass_deposit34 = Mass_deposit
#CMass_deposit34 = CMass_deposit
#Water_cum34 = Water_cum
#Mass_cum34 = Mass_cum
#DryIce_cum34 = DryIce_cum
#CMass_cum34 = CMass_cum

##MY35
#Water_deposit35 = Water_deposit
#CO2_deposit35 = CO2_deposit
#Mass_deposit35 = Mass_deposit
#CMass_deposit35 = CMass_deposit
#Water_cum35 = Water_cum
#Mass_cum35 = Mass_cum
#DryIce_cum35 = DryIce_cum
#CMass_cum35 = CMass_cum

In [ ]:
#get rid of outliers in the data:
#water 
Water_deposit29_B = remove_outliers(Water_deposit29)
Water_deposit30_B = remove_outliers(Water_deposit30)
Water_deposit31_B = remove_outliers(Water_deposit31)
Water_deposit32_B = remove_outliers(Water_deposit32)
Water_deposit33_B = remove_outliers(Water_deposit33)
Water_deposit34_B = remove_outliers(Water_deposit34)
Water_deposit35_B = remove_outliers(Water_deposit35)


#water mass
Mass_deposit29_B = remove_outliers(Mass_deposit29)
Mass_deposit30_B = remove_outliers(Mass_deposit30)
Mass_deposit31_B = remove_outliers(Mass_deposit31)
Mass_deposit32_B = remove_outliers(Mass_deposit32)
Mass_deposit33_B = remove_outliers(Mass_deposit33)
Mass_deposit34_B = remove_outliers(Mass_deposit34)
Mass_deposit35_B = remove_outliers(Mass_deposit35)


#co2
CO2_deposit29_B = remove_outliers(CO2_deposit29)
CO2_deposit30_B = remove_outliers(CO2_deposit30)
CO2_deposit31_B = remove_outliers(CO2_deposit31)
CO2_deposit32_B = remove_outliers(CO2_deposit32)
CO2_deposit33_B = remove_outliers(CO2_deposit33)
CO2_deposit34_B = remove_outliers(CO2_deposit34)
CO2_deposit35_B = remove_outliers(CO2_deposit35)


#co2 mass
CMass_deposit29_B = remove_outliers(CMass_deposit29)
CMass_deposit30_B = remove_outliers(CMass_deposit30)
CMass_deposit31_B = remove_outliers(CMass_deposit31)
CMass_deposit32_B = remove_outliers(CMass_deposit32)
CMass_deposit33_B = remove_outliers(CMass_deposit33)
CMass_deposit34_B = remove_outliers(CMass_deposit34)
CMass_deposit35_B = remove_outliers(CMass_deposit35)

In [ ]:
#fill in gaps that are too long (several Ls's) in MY 28 and 34 with the average of MY 30-33
#water
Water_average = (Water_deposit29_B + Water_deposit32_B+Water_deposit33_B)/3
Water_deposit30_B[:24] = Water_average[:24]
Water_deposit31_B[-32:] = Water_average[-32:]
Water_deposit34_B[:79] = Water_average[:79]

#water mass
Mass_average = (Mass_deposit29_B + Mass_deposit32_B+Mass_deposit33_B)/3
Mass_deposit30_B[:24] = Mass_average[:24]
Mass_deposit31_B[-32:] = Mass_average[-32:]
Mass_deposit34_B[:79] = Mass_average[:79]

#co2
CO2_average = (CO2_deposit29_B + CO2_deposit32_B+CO2_deposit33_B)/3
CO2_deposit30_B[:24] = CO2_average[:24]
CO2_deposit31_B[-32:] = CO2_average[-32:]
CO2_deposit34_B[:79] = CO2_average[:79]

#co2 mass
CMass_average = (CMass_deposit29_B + CMass_deposit32_B+CMass_deposit33_B)/3
CMass_deposit30_B[:24] = CMass_average[:24]
CMass_deposit31_B[-32:] = CMass_average[-32:]
CMass_deposit34_B[:79] = CMass_average[:79]


In [ ]:
#recalculate the cumulative values for those two years

Water_cum29 =  np.cumsum(Water_deposit29_B, axis = 0)
Mass_cum29 =   np.cumsum(Mass_deposit29_B, axis = 0)
DryIce_cum29 = np.cumsum(CO2_deposit29_B, axis = 0)
CMass_cum29 =  np.cumsum(CMass_deposit29_B, axis = 0)

Water_cum30 =  np.cumsum(Water_deposit30_B, axis = 0)
Mass_cum30 =   np.cumsum(Mass_deposit30_B, axis = 0)
DryIce_cum30 = np.cumsum(CO2_deposit30_B, axis = 0)
CMass_cum30 =  np.cumsum(CMass_deposit30_B, axis = 0)

Water_cum31 =  np.cumsum(Water_deposit31_B, axis = 0)
Mass_cum31 =   np.cumsum(Mass_deposit31_B, axis = 0)
DryIce_cum31 = np.cumsum(CO2_deposit31_B, axis = 0)
CMass_cum31 =  np.cumsum(CMass_deposit31_B, axis = 0)

Water_cum32 =  np.cumsum(Water_deposit32_B, axis = 0)
Mass_cum32 =   np.cumsum(Mass_deposit32_B, axis = 0)
DryIce_cum32 = np.cumsum(CO2_deposit32_B, axis = 0)
CMass_cum32 =  np.cumsum(CMass_deposit32_B, axis = 0)

Water_cum33 =  np.cumsum(Water_deposit33_B, axis = 0)
Mass_cum33 =   np.cumsum(Mass_deposit33_B, axis = 0)
DryIce_cum33 = np.cumsum(CO2_deposit33_B, axis = 0)
CMass_cum33 =  np.cumsum(CMass_deposit33_B, axis = 0)


Water_cum34 =   np.cumsum(Water_deposit34_B, axis = 0)
Mass_cum34 =    np.cumsum(Mass_deposit34_B, axis = 0)
DryIce_cum34 =  np.cumsum(CO2_deposit34_B, axis = 0)
CMass_cum34 =   np.cumsum(CMass_deposit34_B, axis = 0)

In [ ]:
#store results in dictionary to save locally
South_results = dict()

#MY 29
South_results['Water_deposit29_B']  = Water_deposit29_B 
South_results['CO2_deposit29_B']    = CO2_deposit29_B   
South_results['Mass_deposit29_B']   = Mass_deposit29_B  
South_results['CMass_deposit29_B']  = CMass_deposit29_B 
South_results['Water_cum29']        = Water_cum29     
South_results['Mass_cum29']         = Mass_cum29      
South_results['DryIce_cum29']       = DryIce_cum29    
South_results['CMass_cum29']        = CMass_cum29     

#MY 30
South_results['Water_deposit30_B']  =  Water_deposit30_B
South_results['CO2_deposit30_B']    =  CO2_deposit30_B  
South_results['Mass_deposit30_B']   =  Mass_deposit30_B 
South_results['CMass_deposit30_B']  =  CMass_deposit30_B
South_results['Water_cum30']        =  Water_cum30    
South_results['Mass_cum30']         =  Mass_cum30     
South_results['DryIce_cum30']       =  DryIce_cum30   
South_results['CMass_cum30']        =  CMass_cum30    

#MY 31
South_results['Water_deposit31_B']  =  Water_deposit31_B
South_results['CO2_deposit31_B']    =  CO2_deposit31_B  
South_results['Mass_deposit31_B']   =  Mass_deposit31_B 
South_results['CMass_deposit31_B']  =  CMass_deposit31_B
South_results['Water_cum31']        =  Water_cum31    
South_results['Mass_cum31']         =  Mass_cum31     
South_results['DryIce_cum31']       =  DryIce_cum31   
South_results['CMass_cum31']        =  CMass_cum31    

#MY 32
South_results['Water_deposit32_B']  =  Water_deposit32_B
South_results['CO2_deposit32_B']    =  CO2_deposit32_B  
South_results['Mass_deposit32_B']   =  Mass_deposit32_B 
South_results['CMass_deposit32_B']  =  CMass_deposit32_B
South_results['Water_cum32']        =  Water_cum32    
South_results['Mass_cum32']         =  Mass_cum32     
South_results['DryIce_cum32']       =  DryIce_cum32   
South_results['CMass_cum32']        =  CMass_cum32    

#MY 33
South_results['Water_deposit33_B']  =  Water_deposit33_B
South_results['CO2_deposit33_B']    =  CO2_deposit33_B  
South_results['Mass_deposit33_B']   =  Mass_deposit33_B 
South_results['CMass_deposit33_B']  =  CMass_deposit33_B
South_results['Water_cum33']        =  Water_cum33    
South_results['Mass_cum33']         =  Mass_cum33     
South_results['DryIce_cum33']       =  DryIce_cum33   
South_results['CMass_cum33']        =  CMass_cum33    

#MY 34
South_results['Water_deposit34_B']  =  Water_deposit34_B
South_results['CO2_deposit34_B']    =  CO2_deposit34_B  
South_results['Mass_deposit34_B']   =  Mass_deposit34_B 
South_results['CMass_deposit34_B']  =  CMass_deposit34_B
South_results['Water_cum34']        =  Water_cum34    
South_results['Mass_cum34']         =  Mass_cum34     
South_results['DryIce_cum34']       =  DryIce_cum34   
South_results['CMass_cum34']        =  CMass_cum34    

#MY 35
South_results['Water_deposit35_B']  =  Water_deposit35_B
South_results['CO2_deposit35_B']    =  CO2_deposit35_B  
South_results['Mass_deposit35_B']   =  Mass_deposit35_B 
South_results['CMass_deposit35_B']  =  CMass_deposit35_B
South_results['Water_cum35']        =  Water_cum35    
South_results['Mass_cum35']         =  Mass_cum35     
South_results['DryIce_cum35']       =  DryIce_cum35   
South_results['CMass_cum35']        =  CMass_cum35 

In [ ]:
#to save the data post analysis:
#with open('South_Snowfall_Results2.pickle', 'wb') as handle:
    #pickle.dump(South_results, handle, protocol=pickle.HIGHEST_PROTOCOL)